In [1]:
import pandas as pd
import numpy as np

from jellyfish import *

import warnings
warnings.filterwarnings('ignore')

# Loading data

In [2]:
cod_post_orig_df = pd.read_csv('interim_data\cod_post_orig_clean.csv')
cod_post_map_df = pd.read_csv('interim_data\cod_post_map_clean.csv')
cod_post_decofre_df = pd.read_csv('interim_data\cod_post_decofre.csv')

In [3]:
cod_post_orig_df.head()

,cod_distrito,cod_concelho,cod_localidade,nome_localidade,num_cod_postal,ext_cod_postal,desig_postal,Distrito,Concelho,cod_postal
0,1,1,249,Alcafaz,3750,11,AGADÃO,Aveiro,Águeda,3750011
1,1,1,250,Caselho,3750,12,AGADÃO,Aveiro,Águeda,3750012
2,1,1,251,Corga da Serra,3750,13,AGADÃO,Aveiro,Águeda,3750013
3,1,1,252,Foz,3750,14,AGADÃO,Aveiro,Águeda,3750014
4,1,1,253,Guistola,3750,15,AGADÃO,Aveiro,Águeda,3750015


In [4]:
cod_post_map_df.head()

,Distrito,Concelho,Freguesia,Alteração RATF,Freguesia Final (Pós RATF)
0,Aveiro,Águeda,Aguada de Cima,Sem alteração,Aguada de Cima
1,Aveiro,Águeda,Fermentelos,Sem alteração,Fermentelos
2,Aveiro,Águeda,Macinhata do Vouga,Sem alteração,Macinhata do Vouga
3,Aveiro,Águeda,Valongo do Vouga,Sem alteração,Valongo do Vouga
4,Aveiro,Águeda,Águeda,Agregação,União das freguesias de Águeda e Borralha


In [5]:
cod_post_decofre_df.head()

,Freguesia,Concelho,Distrito,Error,cod_postal
0,Agadão,Águeda,Aveiro,NaN,3750011
1,Agadão,Águeda,Aveiro,NaN,3750012
2,Agadão,Águeda,Aveiro,NaN,3750013
3,Agadão,Águeda,Aveiro,NaN,3750014
4,Agadão,Águeda,Aveiro,NaN,3750015


Replacing *distrito* and *concelho* with the original ones:

In [6]:
cod_post_dcfre_df = cod_post_orig_df[['Distrito', 'Concelho', 'cod_postal']].join(cod_post_decofre_df.set_index('cod_postal')['Freguesia'], on='cod_postal')

In [7]:
cod_post_dcfre_df.head()

,Distrito,Concelho,cod_postal,Freguesia
0,Aveiro,Águeda,3750011,Agadão
1,Aveiro,Águeda,3750012,Agadão
2,Aveiro,Águeda,3750013,Agadão
3,Aveiro,Águeda,3750014,Agadão
4,Aveiro,Águeda,3750015,Agadão


We see that there are some *freguesias* missing for some postal codes. We will leave them missing for now as an easy solution for this problem has not been found.

In [8]:
cod_post_dcfre_df.query('Freguesia.isna()')

,Distrito,Concelho,cod_postal,Freguesia
46,Aveiro,Águeda,3750328,NaN
47,Aveiro,Águeda,3750332,NaN
156,Aveiro,Águeda,3750354,NaN
288,Aveiro,Águeda,3750553,NaN
429,Aveiro,Águeda,3750757,NaN
...,...,...,...,...
194361,Ilha Terceira,Angra do Heroísmo,9700593,NaN
194570,Ilha Terceira,Praia da Vitória,9760302,NaN
195580,Ilha do Pico,Madalena,9950367,NaN
195669,Ilha do Pico,São Roque do Pico,9940070,NaN


# Preparing data for *join* operation

We can start by indexing the data by *Distrito*, *Concelho* and *Freguesia*:

In [9]:
cod_post_dcfre_df.set_index('Distrito', inplace=True)
cod_post_dcfre_df.set_index('Concelho', inplace=True, append=True)
cod_post_dcfre_df.set_index('Freguesia', inplace=True, append=True)

In [10]:
cod_post_dcfre_df.head()

cod_postal
Distrito Concelho Freguesia            
Aveiro   Águeda   Agadão        3750011
                  Agadão        3750012
                  Agadão        3750013
                  Agadão        3750014
                  Agadão        3750015

In [11]:
cod_post_map_df.set_index('Distrito', inplace=True)
cod_post_map_df.set_index('Concelho', inplace=True, append=True)
cod_post_map_df.set_index('Freguesia', inplace=True, append=True)

In [12]:
cod_post_map_df.head()

Alteração RATF  \
Distrito Concelho Freguesia                           
Aveiro   Águeda   Aguada de Cima      Sem alteração   
                  Fermentelos         Sem alteração   
                  Macinhata do Vouga  Sem alteração   
                  Valongo do Vouga    Sem alteração   
                  Águeda                  Agregação   

                                                     Freguesia Final (Pós RATF)  
Distrito Concelho Freguesia                                                      
Aveiro   Águeda   Aguada de Cima                                 Aguada de Cima  
                  Fermentelos                                       Fermentelos  
                  Macinhata do Vouga                         Macinhata do Vouga  
                  Valongo do Vouga                             Valongo do Vouga  
                  Águeda              União das freguesias de Águeda e Borralha

## Simple join

In [13]:
cod_post_final_df = cod_post_dcfre_df.join(cod_post_map_df)

We see that we are left with a lot of missing values (26483). This means the join was not successful in matching all index entries.

In [14]:
cod_post_final_df.isna().sum()

cod_postal                        0
Alteração RATF                26548
Freguesia Final (Pós RATF)    26548
dtype: int64

## Simplifying the text

We can try removing/replacing some *unimportant* characters in the data, to match possible misspellings of the same name:

In [15]:
def simplify(x):
    if type(x) == str:
        x = x.lower()
        x = x.replace(' da ', '')
        x = x.replace(' do ', '')
        x = x.replace(' de ', '')
        x = x.replace(' dos ', '')
        x = x.replace(' das ', '')
        x = x.replace(' d\' ', '')
        x = x.replace(' e ', '')
        x = x.replace(' ', '')
        x = x.replace('-', '')
        x = x.replace('ã', 'a')
        x = x.replace('à', 'a')
        x = x.replace('â', 'a')
        x = x.replace('á', 'a')
        x = x.replace('é', 'e')
        x = x.replace('ê', 'e')
        x = x.replace('ú', 'u')
        x = x.replace('í', 'i')
        x = x.replace('ó', 'o')
        x = x.replace('ô', 'o')
        x = x.replace('õ', 'o')
        x = x.replace('ç', 'c')
        x = x.replace('\'', '')
        x = x.replace('(', '|')
        x = x.replace(')', '')
        x = x.replace(',', '')
    return x

In [16]:
cod_post_dcfre_df.reset_index(level=['Distrito', 'Concelho', 'Freguesia'], inplace=True)

cod_post_dcfre_df['Distrito_idx'] = cod_post_dcfre_df['Distrito'].apply(simplify)
cod_post_dcfre_df['Concelho_idx'] = cod_post_dcfre_df['Concelho'].apply(simplify)
cod_post_dcfre_df['Freguesia_idx'] = cod_post_dcfre_df['Freguesia'].apply(simplify)

cod_post_dcfre_df.set_index('Distrito_idx', inplace=True)
cod_post_dcfre_df.set_index('Concelho_idx', inplace=True, append=True)
cod_post_dcfre_df.set_index('Freguesia_idx', inplace=True, append=True)

In [17]:
cod_post_map_df.reset_index(level=['Distrito', 'Concelho', 'Freguesia'], inplace=True)

cod_post_map_df['Distrito_idx'] = cod_post_map_df['Distrito'].apply(simplify)
cod_post_map_df['Concelho_idx'] = cod_post_map_df['Concelho'].apply(simplify)
cod_post_map_df['Freguesia_idx'] = cod_post_map_df['Freguesia'].apply(simplify)

cod_post_map_df.set_index('Distrito_idx', inplace=True)
cod_post_map_df.set_index('Concelho_idx', inplace=True, append=True)
cod_post_map_df.set_index('Freguesia_idx', inplace=True, append=True)

We then perform the join again:

In [18]:
cod_post_final_df = cod_post_dcfre_df.join(cod_post_map_df, rsuffix='_map')

We can see that the simplification of the text helped match some more entries. We are left with 21798 entries missing (down from 26483).

In [19]:
cod_post_final_df.isna().sum()

Distrito                          0
Concelho                          0
Freguesia                      1257
cod_postal                        0
Distrito_map                  21863
Concelho_map                  21863
Freguesia_map                 21863
Alteração RATF                21863
Freguesia Final (Pós RATF)    21863
dtype: int64

## Matching *Freguesias* with "(" in the name

Please see that "(" has been replaced with "|" in the text simplification.

In [20]:
cod_post_dcfre_idx_df = cod_post_dcfre_df.index.to_frame()
cod_post_map_idx_df = cod_post_map_df.index.to_frame()

cod_post_final_idx_df = cod_post_final_df.query('Freguesia_map.isna() & Freguesia.notna()').index.to_frame().drop_duplicates()
for dist, conc, freg in zip(cod_post_final_idx_df['Distrito_idx'],
                            cod_post_final_idx_df['Concelho_idx'],
                            cod_post_final_idx_df['Freguesia_idx']):

    for freg_map in cod_post_map_idx_df.loc[dist, conc, :]['Freguesia_idx'].values:
        if '|' in freg_map:
            freg_map_split = freg_map.split('|')
            if freg_map_split[-1] == freg \
               or sum([f in freg for f in freg_map_split]) > 0: # Any parts of freg_map in the freg string
                cod_post_dcfre_idx_df.loc[dist, conc, freg] = cod_post_dcfre_idx_df.loc[dist, conc, freg].replace({'Freguesia_idx': freg}, freg_map)  
        elif '|' in freg:
            freg_split = freg.split('|')
            if freg_split[-1] == freg_map \
               or sum([f in freg_map for f in freg_split]) > 0: # Any parts of freg in the freg_map string
                cod_post_dcfre_idx_df.loc[dist, conc, freg] = cod_post_dcfre_idx_df.loc[dist, conc, freg].replace({'Freguesia_idx': freg}, freg_map)

In [21]:
cod_post_dcfre_idx_df.set_index('Distrito_idx', inplace=True)
cod_post_dcfre_idx_df.set_index('Concelho_idx', inplace=True, append=True)
cod_post_dcfre_idx_df.set_index('Freguesia_idx', inplace=True, append=True)

cod_post_dcfre_df.index = cod_post_dcfre_idx_df.index

In [22]:
cod_post_final_df = cod_post_dcfre_df.join(cod_post_map_df, rsuffix='_map')

This step was a major improvement! Down to 3560 missing from 21798.

In [23]:
cod_post_final_df.isna().sum()

Distrito                         0
Concelho                         0
Freguesia                     1257
cod_postal                       0
Distrito_map                  3360
Concelho_map                  3360
Freguesia_map                 3360
Alteração RATF                3360
Freguesia Final (Pós RATF)    3360
dtype: int64

## Using *Levenshtein* + *Jaro* distances

Using the *Levenshtein + Jaro* distances with a threshold (*2* was empirically chosen) we can match the closest pair of *freguesia* names in the same *distrito* and *concelho*:

In [24]:
cod_post_dcfre_idx_df = cod_post_dcfre_df.index.to_frame()
cod_post_map_idx_df = cod_post_map_df.index.to_frame()

cod_post_final_idx_df = cod_post_final_df.query('Freguesia_map.isna() & Freguesia.notna()').index.to_frame().drop_duplicates()
for dist, conc in zip(cod_post_final_idx_df['Distrito_idx'],
                      cod_post_final_idx_df['Concelho_idx']):
    try:
        freg_arr = cod_post_final_idx_df.loc[dist, conc, :]['Freguesia_idx'].values
        freg_map_arr = cod_post_map_idx_df.loc[dist, conc, :]['Freguesia_idx'].values
        freg_dists = np.zeros([len(freg_arr), len(freg_map_arr)])
        for i, freg in enumerate(freg_arr):
            for j, freg_map in enumerate(freg_map_arr):
                d0 = d1 = d2 = d3 = d4 = 99999
                
                d0 = levenshtein_distance(freg, freg_map) + jaro_distance(freg, freg_map)
                
                d1 = levenshtein_distance(freg.split('|')[0], freg_map) + jaro_distance(freg.split('|')[0], freg_map)
                
                try:
                    d2 = levenshtein_distance(freg.split('|')[1], freg_map) + jaro_distance(freg.split('|')[1], freg_map)
                except:
                    ...

                d3 = levenshtein_distance(freg, freg_map.split('|')[0]) + jaro_distance(freg, freg_map.split('|')[0])
    
                try:
                    d4 = levenshtein_distance(freg, freg_map.split('|')[1]) + jaro_distance(freg, freg_map.split('|')[1])
                except:
                    ...

                leven_dist = min(d0, d1, d2, d3, d4)
                freg_dists[i, j] = leven_dist

    except:
        ...

    for i, freg in enumerate(freg_arr):
        min_loc = np.where(freg_dists[i]==freg_dists[i].min())
        if freg_dists[i].min() < 2:
            cod_post_dcfre_idx_df.loc[dist, conc, freg] = cod_post_dcfre_idx_df.loc[dist, conc, freg].replace({'Freguesia_idx': freg}, freg_map_arr[min_loc][0])

In [25]:
cod_post_dcfre_idx_df.set_index('Distrito_idx', inplace=True)
cod_post_dcfre_idx_df.set_index('Concelho_idx', inplace=True, append=True)
cod_post_dcfre_idx_df.set_index('Freguesia_idx', inplace=True, append=True)

cod_post_dcfre_df.index = cod_post_dcfre_idx_df.index

In [26]:
cod_post_final_df = cod_post_dcfre_df.join(cod_post_map_df, rsuffix='_map')

The use of the distances matched some of the missing values, but not all, since we used a distance threshold. Down to 2281 missing!

In [27]:
cod_post_final_df.isna().sum()

Distrito                         0
Concelho                         0
Freguesia                     1257
cod_postal                       0
Distrito_map                  2281
Concelho_map                  2281
Freguesia_map                 2281
Alteração RATF                2281
Freguesia Final (Pós RATF)    2281
dtype: int64

## Manual data corrections

At this point, we can visually inspect the causes of the missing values:

In [28]:
final_df = cod_post_final_df.set_index('cod_postal')

In [29]:
final_df.query('Freguesia_map.isna() & Freguesia.notna()').drop_duplicates()

,Distrito,Concelho,Freguesia,Distrito_map,Concelho_map,Freguesia_map,Alteração RATF,Freguesia Final (Pós RATF)
cod_postal,,,,,,,,
3020740,Aveiro,Mealhada,Souselas,NaN,NaN,NaN,NaN,NaN
3870501,Aveiro,Murtosa,Beduído,NaN,NaN,NaN,NaN,NaN
3700994,Aveiro,Santa Maria da Feira,São João da Madeira,NaN,NaN,NaN,NaN,NaN
7830109,Beja,Serpa,Vila Nova de São Bento,NaN,NaN,NaN,NaN,NaN
4905009,Braga,Barcelos,Barroselas,NaN,NaN,NaN,NaN,NaN
4765154,Braga,Guimarães,Pedome,NaN,NaN,NaN,NaN,NaN
4805608,Braga,Guimarães,São Vicente de Oleiros,NaN,NaN,NaN,NaN,NaN
4815761,Braga,Guimarães,Tagilde,NaN,NaN,NaN,NaN,NaN
4815756,Braga,Vizela,Conde,NaN,NaN,NaN,NaN,NaN


By examining the data we conclude that it has two different problems:

1.  Some *freguesias* are in the wrong *concelho*/*distrito* because the data comes from the postal code locations so, there are some postal codes in a given *concelho*/*distrito* that belong to a *freguesia* which is located in a different *concelho*/*distrito* (e.g. *Barroselas*).
2.  Some *freguesias* have largelly different names in each of the databases (e.g. *Grovelas São João Evangelista* vs *Grovelas*)

An obvious, automated, approach to solving these problems has not been found. However, they can easily be fixed by hand:

1. Doing a simple internet search of the *freguesias* and checking their *distrito* and *concelho*.
2. Inspecting the databases, for problem 2.

In [30]:
cod_post_dcfre_idx_df = cod_post_dcfre_df.index.to_frame()
cod_post_map_idx_df = cod_post_map_df.index.to_frame()

In [31]:
cod_post_dcfre_idx_df.loc['aveiro', 'mealhada', 'souselas'] = cod_post_dcfre_idx_df.loc['aveiro', 'mealhada', 'souselas'].replace('aveiro', 'coimbra').replace('mealhada', 'coimbra')
cod_post_dcfre_idx_df.loc['aveiro', 'murtosa', 'beduido'] = cod_post_dcfre_idx_df.loc['aveiro', 'murtosa', 'beduido'].replace('murtosa', 'estarreja')
cod_post_dcfre_idx_df.loc['aveiro', 'santamariafeira', 'saojoaomadeira'] = cod_post_dcfre_idx_df.loc['aveiro', 'santamariafeira', 'saojoaomadeira'].replace('santamariafeira', 'saojoaomadeira')
cod_post_dcfre_idx_df.loc['beja', 'serpa', 'vilanovasaobento'] = cod_post_dcfre_idx_df.loc['beja', 'serpa', 'vilanovasaobento'].replace('vilanovasaobento', 'aldeianovasaobento')
cod_post_dcfre_idx_df.loc['braga', 'barcelos', 'barroselas'] = cod_post_dcfre_idx_df.loc['braga', 'barcelos', 'barroselas'].replace('braga', 'vianacastelo').replace('barcelos','vianacastelo')
cod_post_dcfre_idx_df.loc['braga', 'guimaraes', 'pedome'] = cod_post_dcfre_idx_df.loc['braga', 'guimaraes', 'pedome'].replace('guimaraes', 'vilanovafamalicao')
cod_post_dcfre_idx_df.loc['braga', 'guimaraes', 'saovicenteoleiros'] = cod_post_dcfre_idx_df.loc['braga', 'guimaraes', 'saovicenteoleiros'].replace('saovicenteoleiros', 'oleiros')
cod_post_dcfre_idx_df.loc['braga', 'guimaraes', 'tagilde'] = cod_post_dcfre_idx_df.loc['braga', 'guimaraes', 'tagilde'].replace('guimaraes', 'vizela')
cod_post_dcfre_idx_df.loc['braga', 'vizela', 'conde'] = cod_post_dcfre_idx_df.loc['braga', 'vizela', 'conde'].replace({'Concelho_idx': 'vizela'}, 'guimaraes')
cod_post_dcfre_idx_df.loc['braga', 'vizela', 'saojorge|selho'] = cod_post_dcfre_idx_df.loc['braga', 'vizela', 'saojorge|selho'].replace('vizela', 'guimaraes').replace('saojorge|selho', 'selho|saojorge')
cod_post_dcfre_idx_df.loc['coimbra', 'arganil', 'paradela'] = cod_post_dcfre_idx_df.loc['coimbra', 'arganil', 'paradela'].replace('arganil', 'penacova')
cod_post_dcfre_idx_df.loc['coimbra', 'arganil', 'saopedroalva'] = cod_post_dcfre_idx_df.loc['coimbra', 'arganil', 'saopedroalva'].replace('arganil', 'penacova')
cod_post_dcfre_idx_df.loc['coimbra', 'coimbra', 'mirandacorvo'] = cod_post_dcfre_idx_df.loc['coimbra', 'coimbra', 'mirandacorvo'].replace({'Concelho_idx': 'coimbra'}, 'mirandacorvo')
cod_post_dcfre_idx_df.loc['coimbra', 'figueirafoz', 'pelariga'] = cod_post_dcfre_idx_df.loc['coimbra', 'figueirafoz', 'pelariga'].replace('coimbra', 'leiria').replace('figueirafoz', 'pombal')
cod_post_dcfre_idx_df.loc['coimbra', 'gois', 'cabril'] = cod_post_dcfre_idx_df.loc['coimbra', 'gois', 'cabril'].replace('gois', 'pampilhosaserra')
cod_post_dcfre_idx_df.loc['coimbra', 'lousa', 'lavegadas'] = cod_post_dcfre_idx_df.loc['coimbra', 'lousa', 'lavegadas'].replace('lousa', 'vilanovapoiares')
cod_post_dcfre_idx_df.loc['coimbra', 'lousa', 'poiares|santoandre'] = cod_post_dcfre_idx_df.loc['coimbra', 'lousa', 'poiares|santoandre'].replace('lousa', 'vilanovapoiares')
cod_post_dcfre_idx_df.loc['coimbra', 'lousa', 'saomiguelpoiares'] = cod_post_dcfre_idx_df.loc['coimbra', 'lousa', 'saomiguelpoiares'].replace('lousa', 'vilanovapoiares')
cod_post_dcfre_idx_df.loc['coimbra', 'oliveirahospital', 'lagaresbeira'] = cod_post_dcfre_idx_df.loc['coimbra', 'oliveirahospital', 'lagaresbeira'].replace('lagaresbeira', 'lagares')
cod_post_dcfre_idx_df.loc['coimbra', 'penacova', 'cercosa'] = cod_post_dcfre_idx_df.loc['coimbra', 'penacova', 'cercosa'].replace('coimbra', 'viseu').replace('penacova', 'mortagua')
cod_post_dcfre_idx_df.loc['evora', 'borba', 'conceicao'] = cod_post_dcfre_idx_df.loc['evora', 'borba', 'conceicao'].replace('borba', 'alandroal').replace('conceicao', 'alandroal|nossasenhoraconceicao')
cod_post_dcfre_idx_df.loc['evora', 'montemoronovo', 'corticadaslavre'] = cod_post_dcfre_idx_df.loc['evora', 'montemoronovo', 'corticadaslavre'].replace('corticadaslavre', 'corticadas')
cod_post_dcfre_idx_df.loc['ilhaterceira', 'angraheroismo', 'saomateus'] = cod_post_dcfre_idx_df.loc['ilhaterceira', 'angraheroismo', 'saomateus'].replace('saomateus', 'saomateuscalheta')
cod_post_dcfre_idx_df.loc['leiria', 'alcobaca', 'santoonofre'] = cod_post_dcfre_idx_df.loc['leiria', 'alcobaca', 'santoonofre'].replace('alcobaca', 'caldasrainha').replace('santoonofre', 'caldasrainha|santoonofre')
cod_post_dcfre_idx_df.loc['leiria', 'alvaiazere', 'igrejanovasobral'] = cod_post_dcfre_idx_df.loc['leiria', 'alvaiazere', 'igrejanovasobral'].replace('leiria', 'santarem').replace('alvaiazere', 'ferreirazezere')
cod_post_dcfre_idx_df.loc['leiria', 'ansiao', 'abiul'] = cod_post_dcfre_idx_df.loc['leiria', 'ansiao', 'abiul'].replace('ansiao', 'pombal')
cod_post_dcfre_idx_df.loc['leiria', 'peniche', 'miragaia'] = cod_post_dcfre_idx_df.loc['leiria', 'peniche', 'miragaia'].replace('leiria', 'lisboa').replace('peniche', 'lourinha')
cod_post_dcfre_idx_df.loc['lisboa', 'arrudavinhos', 'sapataria'] = cod_post_dcfre_idx_df.loc['lisboa', 'arrudavinhos', 'sapataria'].replace('arrudavinhos', 'sobralmonteagraco')
cod_post_dcfre_idx_df.loc['lisboa', 'lisboa', 'moscavide'] = cod_post_dcfre_idx_df.loc['lisboa', 'lisboa', 'moscavide'].replace({'Concelho_idx': 'lisboa'}, 'loures')
cod_post_dcfre_idx_df.loc['lisboa', 'lisboa', 'sacavem'] = cod_post_dcfre_idx_df.loc['lisboa', 'lisboa', 'sacavem'].replace({'Concelho_idx': 'lisboa'}, 'loures')
cod_post_dcfre_idx_df.loc['lisboa', 'lisboa', 'saocristovao'] = cod_post_dcfre_idx_df.loc['lisboa', 'lisboa', 'saocristovao'].replace('saocristovao', 'saocristovaosaolourenco')
cod_post_dcfre_idx_df.loc['lisboa', 'lourinha', 'monteredondo'] = cod_post_dcfre_idx_df.loc['lisboa', 'lourinha', 'monteredondo'].replace('lourinha', 'torresvedras')
cod_post_dcfre_idx_df.loc['lisboa', 'sobralmonteagraco', 'saoquintino'] = cod_post_dcfre_idx_df.loc['lisboa', 'sobralmonteagraco', 'saoquintino'].replace('saoquintino', 'santoquintino')
cod_post_dcfre_idx_df.loc['lisboa', 'torresvedras', 'santamariasaomiguel'] = cod_post_dcfre_idx_df.loc['lisboa', 'torresvedras', 'santamariasaomiguel'].replace('santamariasaomiguel', 'torresvedras|santamariacastelosaomiguel')
cod_post_dcfre_idx_df.loc['portalegre', 'elvas', 'ajudasaosalvadorsantoildefonso'] = cod_post_dcfre_idx_df.loc['portalegre', 'elvas', 'ajudasaosalvadorsantoildefonso'].replace('ajudasaosalvadorsantoildefonso', 'ajudasalvadorsantoildefonso')
cod_post_dcfre_idx_df.loc['porto', 'lousada', 'santaeulalia'] = cod_post_dcfre_idx_df.loc['porto', 'lousada', 'santaeulalia'].replace('lousada', 'felgueiras').replace('santaeulalia', 'margaride|santaeulalia')
cod_post_dcfre_idx_df.loc['porto', 'marcocanaveses', 'paredes'] = cod_post_dcfre_idx_df.loc['porto', 'marcocanaveses', 'paredes'].replace('paredes', 'paredesviadores')
cod_post_dcfre_idx_df.loc['santarem', 'entroncamento', 'assentiz'] = cod_post_dcfre_idx_df.loc['santarem', 'entroncamento', 'assentiz'].replace('entroncamento', 'riomaior')
cod_post_dcfre_idx_df.loc['setubal', 'grandola', 'azinheirabarros'] = cod_post_dcfre_idx_df.loc['setubal', 'grandola', 'azinheirabarros'].replace('azinheirabarros', 'azinheirabarrossaomamedesadao')
cod_post_dcfre_idx_df.loc['vianacastelo', 'vianacastelo', 'neiva'] = cod_post_dcfre_idx_df.loc['vianacastelo', 'vianacastelo', 'neiva'].replace('neiva', 'casteloneiva')
cod_post_dcfre_idx_df.loc['vianacastelo', 'pontebarca', 'grovelassaojoaoevangelista'] = cod_post_dcfre_idx_df.loc['vianacastelo', 'pontebarca', 'grovelassaojoaoevangelista'].replace('grovelassaojoaoevangelista', 'grovelas')
cod_post_dcfre_idx_df.loc['vianacastelo', 'pontelima', 'arcos'] = cod_post_dcfre_idx_df.loc['vianacastelo', 'pontelima', 'arcos'].replace('arcos', 'saopedroarcos')
cod_post_dcfre_idx_df.loc['vianacastelo', 'vianacastelo', 'vilanovaanha'] = cod_post_dcfre_idx_df.loc['vianacastelo', 'vianacastelo', 'vilanovaanha'].replace('vilanovaanha', 'anha')
cod_post_dcfre_idx_df.loc['vianacastelo', 'vilanovacerveira', 'gandra'] = cod_post_dcfre_idx_df.loc['vianacastelo', 'vilanovacerveira', 'gandra'].replace('vilanovacerveira', 'pontelima')
cod_post_dcfre_idx_df.loc['viseu', 'cinfaes', 'oliveira'] = cod_post_dcfre_idx_df.loc['viseu', 'cinfaes', 'oliveira'].replace('oliveira', 'oliveiradouro')
cod_post_dcfre_idx_df.loc['viseu', 'tondela', 'lajeosadao'] = cod_post_dcfre_idx_df.loc['viseu', 'tondela', 'lajeosadao'].replace('lajeosadao', 'lajeosa')
cod_post_dcfre_idx_df.loc['viseu', 'viseu', 'santamaria'] = cod_post_dcfre_idx_df.loc['viseu', 'viseu', 'santamaria'].replace('santamaria', 'viseu|santamariaviseu')

In [32]:
cod_post_dcfre_idx_df.set_index('Distrito_idx', inplace=True)
cod_post_dcfre_idx_df.set_index('Concelho_idx', inplace=True, append=True)
cod_post_dcfre_idx_df.set_index('Freguesia_idx', inplace=True, append=True)

cod_post_dcfre_df.index = cod_post_dcfre_idx_df.index

In [33]:
cod_post_final_df = cod_post_dcfre_df.join(cod_post_map_df, rsuffix='_map')

We are now left with the same number of missing values as we started with - postal codes for which we do not have a *freguesia* name!

In [34]:
cod_post_final_df.isna().sum()

Distrito                         0
Concelho                         0
Freguesia                     1257
cod_postal                       0
Distrito_map                  1257
Concelho_map                  1257
Freguesia_map                 1257
Alteração RATF                1257
Freguesia Final (Pós RATF)    1257
dtype: int64

# Building the final database

In [35]:
final_df = cod_post_final_df.set_index('cod_postal')

In [36]:
final_df.head()

,Distrito,Concelho,Freguesia,Distrito_map,Concelho_map,Freguesia_map,Alteração RATF,Freguesia Final (Pós RATF)
cod_postal,,,,,,,,
3750011,Aveiro,Águeda,Agadão,Aveiro,Águeda,Agadão,Agregação,"União das freguesias de Belazaima do Chão, Cas..."
3750012,Aveiro,Águeda,Agadão,Aveiro,Águeda,Agadão,Agregação,"União das freguesias de Belazaima do Chão, Cas..."
3750013,Aveiro,Águeda,Agadão,Aveiro,Águeda,Agadão,Agregação,"União das freguesias de Belazaima do Chão, Cas..."
3750014,Aveiro,Águeda,Agadão,Aveiro,Águeda,Agadão,Agregação,"União das freguesias de Belazaima do Chão, Cas..."
3750015,Aveiro,Águeda,Agadão,Aveiro,Águeda,Agadão,Agregação,"União das freguesias de Belazaima do Chão, Cas..."


In [37]:
final_df.to_csv('output_data\cod_post_freg_matched.csv')